In [2]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


In [ ]:
from stackable_pipe import pipe, slot, ring, socket
from solid import cylinder, translate, rotate


def spin_around_tower(obj, count=4, alpha=45,h=160):
    angles = range(0, 360, int(360/count))
    # drehen und dann kippen
    return translate((0, 0, h/2))([rotate((alpha, 0, z))(obj) for z in angles])


def tower_socket(d=60, w=4):
    r = d/2
    base = rotate((0, 0, 15))(socket(d=d, w=w)) + ring(d1=d, h=d+2*w, w=w/2)
    transition = ring(r2=r+w*3, r1=r, h=w*3, w=w*4) - cylinder(h=w*3, d=d)

    return base # + transition


def aero_tower(w=4,h=160,d=160,pipe_count=5, alpha=45):
    tower = pipe(d=d, h=h, w=w)
    outline = cylinder(d=d, h=d, segments=60)
    holes = spin_around_tower(cylinder(d=60, h=h, segments=60), count=pipe_count, alpha=alpha)
    sockets = spin_around_tower(translate((0, 0, d/2))(tower_socket(w=w)), h=d, count=pipe_count, alpha=alpha)
    
    holes = translate((0,0,-h/2))(holes)
    sockets = translate((0,0,-h/2))(sockets)

    aero_tower = tower - holes
    aero_tower += sockets
    aero_tower *= outline
    return aero_tower


render(aero_tower(), outfile='./outfiles/aero_tower.stl')


In [3]:
from stackable_pipe import pipe, slot, ring, socket, cube
from solid import cylinder, translate, rotate

def bottom_grid(d=160, fogger_diameter=100, slots=5):
    plate = cylinder(h=2, d=d, segments=60)
    hole = translate((fogger_diameter/2, -5, 0))(cube(((d-fogger_diameter)/2,10,2)))

    holes = sum([rotate((0, 0, r))(hole) for r in range(0, 360, 360//slots)])

    return plate - holes

def aero_tower_fogger_inlet(w=4,d=160, fogger_height=80, fogger_diameter=100, fogger_gap=1, fog_slots=5):
    h = fogger_height + 7*w + 2

    tower = pipe(d=d, h=h, w=w)
    
    inlet = cylinder(segments=60, d=fogger_diameter+w, h=fogger_height + 2)\
         - cylinder(segments=60, d=fogger_diameter+fogger_gap, h=fogger_height + 2)

    return tower + inlet + bottom_grid(d=d-4*w, slots=10, fogger_diameter=fogger_diameter+w)

render(aero_tower_fogger_inlet(), outfile='./outfiles/aero_tower_fogger_inlet.stl')

/home/gitpod/.pyenv/versions/3.8.12/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [13]:
from stackable_pipe import pipe, slot, ring, socket, adapter
from solid import cylinder, translate, rotate, cube, hull


def adapter_bottom_grid(d=60, w=4):
    plate = cylinder(h=2, d=d-4*w)
    hole = cylinder(h=2, d=6)

    holes = sum([rotate((0, 0, r))(translate((x, 0, 0))(hole))
                for r in range(0, 360, 360//8) for x in [0, 9, 16]])

    return plate - holes


def aero_tower_plant_cup(w=4, d=60, h=90, gap=2):
    bottom = adapter(d=d, w=w)
    grid = adapter_bottom_grid(d=d, w=w)
    pipe = ring(d1=d-2*w, w=w, h=h-4*w, dz=4*w)
    outline = cylinder(d=d-2*(w+gap), h=h, segments=100)

    return (bottom + grid + pipe) * outline


render(aero_tower_plant_cup(), outfile='./outfiles/aero_tower_plant_cup.stl')


/home/gitpod/.pyenv/versions/3.8.12/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [ ]:
d = 160
h=160
w=4
pipes=1
cups = spin_around_tower(translate((0,0, d/2-w))(rotate((0,0,180))(aero_tower_cap())), count=pipes)
cups = translate((0,0,-h/2))(cups)

showcase = aero_tower(pipe_count=pipes) + cups
render(showcase - cube(200))




# render(sum([translate((0,0,z))(showcase) for z in range(0,h*6,h-(4*w))]), outfile='./outfiles/showcase.stl')

In [17]:
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


from stackable_pipe import socket
from solid import cylinder, sphere, translate, cube

def tank(size=180, wall=2, border_radius=30, socket_d=160, socket_w=4):
    top_height = size-socket_d+socket_w*3
 
    body = cylinder(d=size, h=size-top_height, segments=100)\
        - translate((0,0,wall))(cylinder(d=(size-(wall*2)), h=size, segments=100))

    top = translate((0,0,size-socket_d))(socket(d=socket_d, w=socket_w))\
        + cylinder(d1=size, d2=socket_d, h=size-socket_d, segments=100)\
        - cylinder(d1=size-(wall*2), d2=socket_d-(wall*2), h=size-socket_d, segments=100) 

    return body + translate((0,0,size-top_height))(top)


render(tank(), outfile='./outfiles/tank.stl')